## change to code and run on colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.10
!pip install wandb
import tensorflow as tf
print(tf.__version__)

In [2]:
import warnings 
warnings.filterwarnings(action='ignore')
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import inception_resnet_v2, nasnet
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback
import wandb
import tensorflow as tf
tf.__version__

'2.10.0'

# Activate this cell when you need
# split data
ori_flood = '../data/binary/flood'
ori_normal = '../data/binary/normal'

img_flood = glob(ori_flood+'/*.jpg')
img_normal = glob(ori_normal+'/*.jpg')

len_flood=len(img_flood)
print('len_flood : ', len_flood)
len_normal=len(img_normal)
print('len_normal : ', len_normal)

train_dir = '../data/binary/train'
valid_dir = '../data/binary/valid'

# copy flood img
for i, img in enumerate(tqdm(img_flood)):
    img_name = img.split('/')[-1]
    src = img
    if i < len_flood * 0.7 :
        dst = os.path.join(train_dir+'/flood', img_name)
    else :
        dst = os.path.join(valid_dir+'/flood', img_name)
    shutil.copyfile(src,dst)

# copy normal img
for i, img in enumerate(tqdm(img_normal)):
    img_name = img.split('/')[-1]
    src = img
    if i < len_normal * 0.7 :
        dst = os.path.join(train_dir+'/normal', img_name)
    else :
        dst = os.path.join(valid_dir+'/normal', img_name)
    shutil.copyfile(src,dst)

# Make DataFrame

In [16]:
# flood : 1, normal : 0
flood_df = pd.DataFrame({'path' : glob('../data/binary/flood/*.jpg'),
                        'label' : '1'})
print('flood len : {}'.format(len(flood_df)))
normal_df = pd.DataFrame({'path' : glob('../data/binary/normal/*.jpg'),
                         'label': '0'})
print('normal len : {}'.format(len(normal_df)))

df = pd.concat([flood_df, normal_df], axis=0)

#shuffle
df = df.sample(frac=1)
display(df.head())

#split
x_train, x_test, y_train, y_test = train_test_split(df['path'],
                                                   df['label'],
                                                   stratify=df['label'],
                                                   test_size=0.2)
train_df = pd.DataFrame({'path':x_train,
                        'label':y_train})
valid_df = pd.DataFrame({'path':x_test,
                        'label':y_test})
train_df.head()

flood len : 489
normal len : 867


,path,label
478,../data/binary/normal/frame5.jpg,0
125,../data/binary/normal/616.jpg,0
786,../data/binary/normal/도로261.jpg,0
283,../data/binary/flood/527.jpg,1
463,../data/binary/normal/frame264.jpg,0


,path,label
593,../data/binary/normal/frame17.jpg,0
329,../data/binary/normal/frame327.jpg,0
341,../data/binary/flood/290.jpg,1
272,../data/binary/normal/frame325.jpg,0
346,../data/binary/flood/247.jpg,1


# Config

In [17]:
# train_dir = '/content/drive/MyDrive/융합캡스톤디자인/binary/train'
# valid_dir = '/content/drive/MyDrive/융합캡스톤디자인/binary/valid'
# train_dir = '../data/binary/train'
# valid_dir = '../data/binary/valid'

CLASSES = ['normal', 'flood']
NasNet = 'NasNet'
Inception = 'Inception'
target_size = {NasNet : (331,331),
               Inception : (299,299)}
EPOCHS = 1000

sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'early_terminate' : {
        'type': 'hyperband',
        'max_iter': 27,
        's': 2
    },
    'parameters': {
        'pretrain_net': {
            'values': [Inception, NasNet]
        },
        'batch_size': {
            'values': [20, 50]
        },
        'dropout': {
            'values': [0.2, 0.5]
        },
        'dense': {
            'values': [32, 256, 1024]
        },
        'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-5]
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop']
        },
        'activation': {
            'values': ['relu', 'elu', 'selu', 'softmax']
        }
    }
}

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255,
                              rotation_range=30,
                              shear_range=0.2,
                              zoom_range=0.3,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode = 'nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
def train() :
    tf.keras.backend.clear_session()
    config_defaults = {
        'pretrain_net': Inception,
        'epochs' : EPOCHS,
        'batch_size': 50,
        'dropout' : 0.5,
        'learning_rate' : 1e-3,
        'activation': 'relu',
        'optimizer': 'adam',
        'dense':1024
    }
    
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_generator = train_datagen.flow_from_dataframe(train_df, 
                                               x_col='path',
                                               y_col='label',
                                               target_size=target_size[config.pretrain_net],
                                               class_mode='binary',
                                               batch_size=config.batch_size)
    valid_generator = valid_datagen.flow_from_dataframe(valid_df,
                                               x_col='path',
                                               y_col='label',
                                               target_size=target_size[config.pretrain_net],
                                               class_mode='binary',
                                               batch_size=config.batch_size)    
    
    if config.pretrain_net == Inception :
        base_model = inception_resnet_v2.InceptionResNetV2(
            weights='imagenet',
            include_top = False,
            input_shape = target_size[config.pretrain_net]+(3,)
        )
    elif config.pretrain_net == NasNet :
        base_model= nasnet.NASNetLarge(
              weights='imagenet',
              include_top=False,
              input_shape = target_size[config.pretrain_net]+(3,)
             )
    base_model.trainable = False 

    inputs = layers.Input(shape=target_size[config.pretrain_net]+(3,))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(config.dropout)(x)
    x = layers.Dense(config.dense, activation=config.activation)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, 
                  outputs=outputs,
                  name='binary_flood')    
    
    if config.optimizer=='sgd':
        optimizer = SGD(learning_rate=config.learning_rate)
    elif config.optimizer=='rmsprop':
        optimizer = RMSprop(learning_rate=config.learning_rate)
    elif config.optimizer=='adam':
        optimizer = Adam(learning_rate=config.learning_rate)


    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])    
    
    _ = model.fit(train_generator,
          validation_data=valid_generator,
          verbose=1,
          epochs=config.epochs,
          callbacks=[WandbCallback()],
          steps_per_epoch=len(train_df)//config.batch_size)

In [20]:
sweep_id = wandb.sweep(sweep_config, project='flood24')

Create sweep with ID: 67tzw27t
Sweep URL: https://wandb.ai/hkleee/flood24/sweeps/67tzw27t


In [21]:
# 67tzw27t
wandb.agent(sweep_id, train) 

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Waiting for W&B process to finish... (success).
wandb: Synced lively-violet-6: https://wandb.ai/hkleee/2022-2-SCS4031-9to6-Model_image_classification/runs/o0j172bj
wandb: Synced 6 W&B file(s), 1 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20221020_150715-o0j172bj/logs
wandb: Agent Starting Run: q27oeyaw with config:
wandb: 	activation: softmax
wandb: 	batch_size: 50
wandb: 	dense: 1024
wandb: 	dropout: 0.2
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: 	pretrain_net: NasNet


Found 1084 validated image filenames belonging to 2 classes.
Found 272 validated image filenames belonging to 2 classes.
Epoch 1/10


2022-10-20 15:24:43.991156: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.6341 - accuracy: 0.8133

2022-10-20 15:26:41.755625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

INFO:tensorflow:Assets written to: /Users/hkleeeee/Desktop/Flooding24/2022-2-SCS4031-9to6/Model/image_classification/wandb/run-20221020_162257-kyte58l7/files/model-best/assets
wandb: Adding directory to artifact (/Users/hkleeeee/Desktop/Flooding24/2022-2-SCS4031-9to6/Model/image_classification/wandb/run-20221020_162257-kyte58l7/files/model-best)... Done. 0.5s


54/54 [==============================] - 91s 2s/step - loss: 0.2578 - accuracy: 0.8872 - val_loss: 0.1760 - val_accuracy: 0.9301
Epoch 5/10


wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x16da88790> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
# TODO 
# visualize result of inference of validate images